In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = "XXXX"
client = OpenAI()
model_list = client.models.list()
for model in model_list.data:
    print(model.id)

whisper-1
tts-1
dall-e-2
tts-1-hd-1106
tts-1-hd
gpt-4-turbo-2024-04-09
gpt-4-turbo
gpt-3.5-turbo-1106
dall-e-3
gpt-4-0125-preview
gpt-4-turbo-preview
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-16k
gpt-4-1106-preview
gpt-3.5-turbo-16k-0613
babbage-002
gpt-4o-2024-05-13
gpt-3.5-turbo-0613
gpt-4
gpt-4-0613
gpt-3.5-turbo-0125
tts-1-1106
gpt-3.5-turbo
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-0301
text-embedding-ada-002
davinci-002
gpt-4o


In [3]:
import base64
import requests

from openai import OpenAI

In [5]:
def encode_image(image_path):
    with open(image_path,"rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')



In [21]:
#base64_image=encode_image("C://Users//prohr//Downloads//wolf_testing.jpg")

In [24]:
# Path to your image
image_path = "C://Users//prohr//Downloads//wolf_testing.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)


In [32]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://media.istockphoto.com/id/1208985997/photo/portrait-of-grey-wolf-in-the-forest.jpg?s=1024x1024&w=is&k=20&c=f0aTQESyIVjFamM8bDUiaSdCchWPglPJ6hAi4lRUgkc=",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

In [33]:
print(response.choices[0].message.content)

The image shows a wolf standing in what appears to be a natural outdoor setting. The wolf has a thick, multi-colored coat primarily containing shades of gray and brown. Its ears are perked up, and it seems to be looking attentively to the side. The background is blurred, indicating a forested or wilderness environment. The image appears to be sourced from iStock, with credit given to AB Photography.


In [38]:
from IPython.display import display, Image, Audio

import cv2
import base64
import time

In [58]:
video = cv2.VideoCapture("C://Users//prohr//Downloads//SampleVideo_1280x720_2mb.mp4")

In [59]:
base64frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64frames.append(base64.b64encode(buffer).decode("utf-8"))

video.release()
print(len(base64frames), "frames read.")

337 frames read.


In [62]:
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames of a video. Create a short voiceover script in the style of Amitabh Bachchan in Hindi. Only include the narration.",
            *map(lambda x: {"image": x, "resize": 768}, base64frames[0::10]),
        ],
    },
]

In [63]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=PROMPT_MESSAGES,
    max_tokens=500,
)

print(response.choices[0])

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='[अमिताभ बच्चन की गहरी आवाज में]\n\n"इस शांत और सुकून भरी वादी में, जहां हरियाली अपनी पूरी शबाब पर है, एक प्यारा सा खरगोश, अपनी गुफा के दरवाजे पर बैठा, आराम का आनंद ले रहा है। देखिए उसकी मासूम आँखों में चंचलता की छवि, उसके शरीर की मुद्रा जैसे कह रही हो, \'आज का दिन सिर्फ और सिर्फ आराम के लिए है।\'"\n\n"धीरे-धीरे खुलती उसकी आँखें, एक अनजाने की उम्मीद में, फिर सुस्ती से पलकें झपकाते हुए। क्या यह नजारा कुछ किस्सा सुनाने को आतुर है? या यह खरगोश अपने स्वप्निल संसार में खोया हुआ है।"\n\n"और अब, वो धूप का आनंद लेत', role='assistant', function_call=None, tool_calls=None))


In [64]:
speech_file_path = "Bachchan.mp3"
audio_response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input=response.choices[0].message.content
)

In [65]:
audio_response.stream_to_file(speech_file_path)
Audio(speech_file_path, autoplay=True)

C:\Users\prohr\AppData\Local\Temp\ipykernel_11148\4130749382.py:1: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  audio_response.stream_to_file(speech_file_path)
